In [1]:
from pathlib import Path

from src.config import DOCUMENTS_DIR, TEST_DIR
from src.llm_query import call_llm
from src.retriever import jaccard_retriever, get_fittest_chunk, tfidf_retriever
from src.chunk import truncate_on_h2, find_chunk_indices_with_title
from src.prompt import build_prompt
import requests
import json


path = Path(DOCUMENTS_DIR)


2025-02-13 22:25:38.350 | INFO     | src.config:<module>:8 - PROJ_ROOT path is: E:\cours\nlp\nlp_rag


In [3]:
texts = []
for filename in path.glob("*.md"):
    with open(filename, encoding="utf-8") as f:
        texts.append(f.read())


## trucate file into chunks

In [4]:
chunks = truncate_on_h2(texts)

In [5]:
chunks

[{'title': 'burn::tensor# Module activation',
  'chunk': 'burn::tensor# Module activation|||: burn::tensor# Module activation\nExpand descriptionThe activation module.\n',
  'index': 0},
 {'title': 'burn::tensor# Module activation',
  'chunk': 'burn::tensor# Module activation|||:  Functions\n- geluApplies the Gaussian Error Linear Units function as described in the paper Gaussian Error Linear Units (GELUs).\n- hard_sigmoidApplies the hard sigmoid function\n- leaky_reluApplies the leaky rectified linear unit function.\n- log_sigmoidApplies the log sigmoid function.\n- log_softmaxApplies the log softmax function on the input tensor along the given dimension.\n- mishApplies the Mish function as described in the paper in Mish: A Self Regularized Non-Monotonic Neural Activation Function.\n- preluApplies Parametric ReLu activation function as described in the paper Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet Classification.\n` PReLu(x) = max(0,x) + \\alpha * 

In [24]:
chunk_tensors_id = find_chunk_indices_with_title(chunks, "burn::tensor# Function split")

In [25]:
tensor_chunk_list = [chunks[i] for i in chunk_tensors_id]

In [26]:
tensor_chunk_list

[{'title': 'burn::tensor# Function split',
  'chunk': 'burn::tensor# Function split|||: burn::tensor# Function split\n`pub fn split where\n    B: Backend,\n    K: TensorKind \n`Expand descriptionSplits the tensor along the given dimension into equally sized chunks (if possible)\nwith size `split_size\n`. Last chunk will be smaller if the tensor size along the given\ndimension `dim\n` is not divisible by `split_size\n`.\n',
  'index': 334},
 {'title': 'burn::tensor# Function split',
  'chunk': 'burn::tensor# Function split|||:  Arguments\n\n- `tensor\n` - The tensor.\n\n- `split_size\n` - The size of a single chunk.\n\n- `dim\n` - The dimension along which to split the tensor.\n\n',
  'index': 335},
 {'title': 'burn::tensor# Function split',
  'chunk': 'burn::tensor# Function split|||:  Returns\n\nA vector of tensors.\n',
  'index': 336},
 {'title': 'burn::tensor# Function split',
  'chunk': 'burn::tensor# Function split|||:  Remarks\n\nThis (and the following) are fallback solutions th

In [ ]:
with open(TEST_DIR / 'questions_evaluation.json', 'r') as f:
    eval_json = json.load(f)
eval_json

In [ ]:
fittest_doc_id = tfidf_retriever(chunks, eval_json[0]['question'])[0]
fittest_doc = chunks[fittest_doc_id]

In [ ]:
fittest_doc

In [ ]:
prompt = build_prompt(eval_json[0]['question'], fittest_doc)

In [ ]:
response = call_llm(prompt ,llm="llama2")

In [ ]:
response

In [ ]:
from src.evaluation import evaluate_rag

In [ ]:
evaluate_rag(eval_json, tfidf_retriever, call_llm, chunks)